# Ch6.Agents - 자율적으로 외부와 상호 작용해 언어 모델의 한계를 뛰어넘기
- 1. 외부와 상호작용하면서 자율적으로 행동하는 Agents
- 2. Tool을 추가해 Agent가 할 수 있는 일을 늘리기
- 3. Tool을 직접 제작해 기능 확장하기
- 4. Retrievers를 사용해 문장을 검색하는 Tool 만들기
- 5. 문맥에 맞게 답변하는 에이전트 만들기

## 1. 외부와 상호작용하면서 자율적으로 행동하는 Agents
> 길찾기는 네이버지도앱을 켜고, 지하철은 카카오지하철을 활용하듯, Agents 모듈을 사용하면 LLM도 작업에 맞는 도구를 선택하여 실행 할 수 있다.

Agents 모듈에는 두가지 하위모듈이 있음
- Tool : 언어모델만으로 할 수 없는 일을 할 수 있도록 만들어주는 도구
    - LLMMath : 언어모델이 취약한 계산을 위한 Tool
    - Requests : 지정된 URL로 요청을 보냄. 웹사이트 정보를 가져오거나, 공개된 API에서 정보 가져옴
    - File System Tools : PC 내 파일에 접근해 지정된 경로의 파일을 읽고 쓸 수 있음. 실행결과를 저장하거나, 파일에 저장된 정보 불러옴
    - SerpAPI : 구글이나 야후 검색을 API로 하는 SerpAPI라는 웹서비스와 연동. 언어모델이 알지못하는 정보를 구글링하여 사용
- Agent : 적절한 Tool을 선택하고 다음 단계(ReAct기법)의 처리를 수행하는 주체 
    1. 사용자로부터 작업을 받는다
    2. 준비된 Tool 중에서 어떤 Tool을 사용할지, 어떤 정보를 입력할 지 결정
    3. Tool을 사용해 결과를 얻음
    4. 3에서 얻은 결과를 통해 과업이 달성되고 있는지 확인
    5. 에이전트가 작업을 완료했다고 판단할 수 있을 때까지 2~4를 반복
> 단순히 Tool을 조작하는 것이 아니다. 어떤 Tool을 어떻게 사용하면 될지 고민/실행하고, 결과 검증까지 하는 것이 Agent의 가장 큰 가치이자 특징

### 주어진 URL에서 정보를 얻는 Agent 생성

In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()
os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo")

tools = load_tools(  #← LangChain에 준비된 Tool을 로드
    [
        "requests"  #← 특정 URL의 결과를 얻는 Tool인 requests를 로드
    ],
    allow_dangerous_tools=True,
    handle_parsing_errors =True
)

agent = initialize_agent(  #← Agent를 초기화
    tools=tools,  #← Agent가 사용할 수 있는 Tool의 배열을 설정
    llm=chat,  #← Agent가 사용할 언어 모델을 지정
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #←ReAct 방식으로 작동하게 설정
    verbose=True,  #← 실행 중 로그를 표시
)

result = agent.run("""아래 URL에 접속해 도쿄의 날씨를 검색해 한국어로 답하세요.
https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json
""")

print(f"실행 결과: {result}")




> Entering new AgentExecutor chain...
Question: Please access the URL below and search for the weather in Tokyo, then answer in Korean.
https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json

Thought: I will use the requests_get tool to access the provided URL and retrieve the weather information for Tokyo.

Action:
```
{
  "action": "requests_get",
  "action_input": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json"
}
```


Observation: {"publishingOffice":"気象庁","reportDatetime":"2024-05-13T16:43:00+09:00","targetArea":"東京都","headlineText":"","text":"　オホーツク海の低気圧からのびる前線が北日本から東日本を通って、日本の南にのびています。\n\n　東京地方は、雨となっています。\n\n　１３日は、前線が東日本を通過する見込みです。このため、雨で夜のはじめ頃まで雷を伴う所がありますが、夜遅くは曇りとなるでしょう。伊豆諸島では雨で雷を伴い非常に激しく降る所がある見込みです。\n\n　１４日は、北日本に中心を持つ高気圧に覆われますが、気圧の谷や湿った空気の影響を受ける見込みです。このため、曇りで未明は雨の降る所がありますが、昼過ぎから晴れるでしょう。伊豆諸島では雨で雷を伴い激しく降る所がある見込みです。\n\n【関東甲信地方】\n　関東甲信地方は、雨となっています。\n\n　１３日は、前線が東日本を通過する見込みです。このため、雨で雷を伴い非常に激しく降る所があるでしょう。\n\n　１４日は、北日本に中心を持つ高気圧に覆われますが、気圧の谷や

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: 

In [14]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()


chat = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    anthropic_api_key=os.getenv("Anthropic_API_KEY")
)

tools = load_tools(  #← LangChain에 준비된 Tool을 로드
    [
        "requests"  #← 특정 URL의 결과를 얻는 Tool인 requests를 로드
    ],
    allow_dangerous_tools=True
)

agent = initialize_agent(  #← Agent를 초기화
    tools=tools,  #← Agent가 사용할 수 있는 Tool의 배열을 설정
    llm=chat,  #← Agent가 사용할 언어 모델을 지정
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #←ReAct 방식으로 작동하게 설정
    
    verbose=True  #← 실행 중 로그를 표시
)

result = agent.run("""아래 URL에 접속해 도쿄의 날씨를 검색해 한국어로 답하세요.
https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json
""")

print(f"실행 결과: {result}")




> Entering new AgentExecutor chain...
Thought: 이 URL에서 도쿄의 날씨 정보를 가져올 수 있는지 확인해야 합니다.

Action:
```json
{
  "action": "requests_get", 
  "action_input": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json"
}
```


Observation: {"publishingOffice":"気象庁","reportDatetime":"2024-05-13T16:43:00+09:00","targetArea":"東京都","headlineText":"","text":"　オホーツク海の低気圧からのびる前線が北日本から東日本を通って、日本の南にのびています。\n\n　東京地方は、雨となっています。\n\n　１３日は、前線が東日本を通過する見込みです。このため、雨で夜のはじめ頃まで雷を伴う所がありますが、夜遅くは曇りとなるでしょう。伊豆諸島では雨で雷を伴い非常に激しく降る所がある見込みです。\n\n　１４日は、北日本に中心を持つ高気圧に覆われますが、気圧の谷や湿った空気の影響を受ける見込みです。このため、曇りで未明は雨の降る所がありますが、昼過ぎから晴れるでしょう。伊豆諸島では雨で雷を伴い激しく降る所がある見込みです。\n\n【関東甲信地方】\n　関東甲信地方は、雨となっています。\n\n　１３日は、前線が東日本を通過する見込みです。このため、雨で雷を伴い非常に激しく降る所があるでしょう。\n\n　１４日は、北日本に中心を持つ高気圧に覆われますが、気圧の谷や湿った空気の影響を受ける見込みです。このため、晴れや曇りで、関東地方の沿岸部では、はじめ雨の降る所があるでしょう。伊豆諸島では、前線の影響で、雨のち曇りで雷を伴い激しく降る所がある見込みです。\n\n　関東地方と伊豆諸島の海上では、うねりを伴い、１３日はしけとなり、１４日は波が高いでしょう。また、所々で霧が発生しています。船舶は高波や視程障害に注意してください。"}
Thought:Final Answer: 東京地方は現在雨となっています。13日は前線の影響で雨や

> ChatGPT3.5로는 Tool을 끝까지 구현이 어려웠으나, Claud3 Sonnet은 구현 가능

Agent의 종류에 대해
```
에이전트는 툴을 사용하여 특정 태스크를 달성한다. 에이전트를 조작하는 방법은 다양하며,
그에 따라 툴의 조작 방법도 다르다
```
- AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION : 대화형식의 상호작용을 특징으로 하며, Chat Models모듈과 ReAct방법을 사용해 작동하는 Agent
- AgentType.ZERO_SHOT_REACT_DESCRIBTION : LLMs 모듈 사용을 전제로 하는 에이전트
- AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION : 여러 입력을 가진 툴을 사용하는 에이전트

## 2. Tool을 추가해 Agent가 할 수 있는 일을 늘리기
> 파일 저장을 할 수 있는 Tool과 구글검색을 할 수 있는 Tool을 추가하고, 결과를 파일로 저장하는 기능을 추가해본다.

- 1. [SerpAPI](https://serpapi.com) 에 가입하고 API를 저장하자. FreePlan 월100회 무료이용
- 2. .env에  SERP_API_KEY=xxxxx을 입력하여 API_KEY저장
- 3. 가상환경에 pip install google-search-results

In [17]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_anthropic import ChatAnthropic
from langchain.tools.file_management import WriteFileTool  #←파일 쓰기를 할 수 있는 Tool을 가져오기
from dotenv import load_dotenv
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

# chat = ChatOpenAI(model="gpt-3.5-turbo")
chat = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    anthropic_api_key=os.getenv("Anthropic_API_KEY"),

)

tools = load_tools(
    [
        "requests_get",
        "serpapi" #←serpapi를 추가
    ],
    llm=chat,
    allow_dangerous_tools=True,
    serpapi_api_key=os.getenv("SERP_API_KEY")
)

tools.append(WriteFileTool( #←파일 쓰기를 할 수 있는 Tool을 추가
    root_dir="./"
))

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #←Agent 유형을 변경
    handle_parsing_errors=True,
    verbose=True
)
# result = agent.run("캘리포니아 해변에서 여름에 할 일을 검색해 result_GPT.txt 파일에 한국어로 저장하세요.") #← GPT

search_result = agent.run("캘리포니아 해변에서 여름에 할 일을 검색해 한국어로 답하세요.")#← Claud3
result = agent.run(f"다음 텍스트를 result_claud3.txt 파일에 저장하세요: file_path='./result/result_claud3.txt', text='{search_result}'")


print(f"실행 결과: {result}")




> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Search",
  "action_input": "Things to do at California beaches in summer"
}
```


Observation: ['Beaches in California · 1. La Jolla Cove · 2. Santa Monica State Beach · 3. Venice Beach · 4. Moonstone Beach · 5. La Jolla Shores Park · 6.', 'Find fun things to do on the coast of California. Discover shipwrecks, butterfly groves, hot springs, beaches you can drive on, and much more.', "If you're looking for things to do in the summer in L.A., we've put together 75 fun ideas, from a day at the beach to a night at a free concert.", 'Your Ultimate Guide to the Best Places to Explore in California in Summer · Big Sur · Sequoia National Forest · Laguna Beach · Mendocino · San Diego · Mammoth Lakes.', '1. Walk, bike, or ride across the Golden Gate Bridge · 2. Explore art in Oakland · 3. Step back in time in Sacramento · 4. Geek out in Silicon Valley · 5. Say hello ...', 'Find things to do this summer in The Bay Area. Enjoy 

- 복수의 tool 활용시 STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION을 이용
> Tool 에는 단일 입력으로 작동하는 것과 여러개의 입력이 필요한 것이 있다. CHAT_ZERO_SHOT_REACT_DESCRIPTION에서는 단일입력의 tool만 받았었다.
위에서는 1. 텍스트를 생성 후 2. WriteFileTool로 저장 하므로 두개의 tool을 실행하게 되어 STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION


In [25]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_anthropic import ChatAnthropic
from langchain.tools.file_management import WriteFileTool  #←파일 쓰기를 할 수 있는 Tool을 가져오기
from dotenv import load_dotenv
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

chat = ChatOpenAI(model="gpt-3.5-turbo")

tools = load_tools(
    [
        "requests_get",
        "serpapi" #←serpapi를 추가
    ],
    llm=chat,
    allow_dangerous_tools=True,
    serpapi_api_key=os.getenv("SERP_API_KEY")
)

tools.append(WriteFileTool( #←파일 쓰기를 할 수 있는 Tool을 추가
    root_dir="./result/",
))

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_FUNCTIONS, # 제대로 동작하지 않으면 OPENAI_MULTI_FUNCTIONS
    verbose=True
)
result = agent.run("캘리포니아 해변에서 여름에 할 일을 검색해 result_GPT.txt 파일에 한국어로 저장하세요.") #← GPT

print(f"실행 결과: {result}")



> Entering new AgentExecutor chain...

Invoking: `Search` with `캘리포니아 해변 여름 활동`


['와인 테스팅을 하거나 카약을 빌리는 등 할거리가 많고 자연 공원도 많이 있어서 바깥 활동을 좋아하는 사람들에게는 하루를 보내는 데에 최적의 해변 ...', '신고하기. \u200b. 안녕하세요^^. 이번주부터 포스트 <스타에디터2>로 활동 ... 해변들 중. 너무나 예뻐서 제 마음의 No.1 해변이 된. 샌 ... 캘리포니아의 여름 ...', '세계에서도 유명한 샌디에이고 동물원. 코스코에서 표를 사서 가면 저렴하고 가족들과 가기 좋은곳. 가을 이나 봄에 가는걸 추천하며 여름에는 너무 더워 힘듭니다.', '캘리포니아가 여름 더위로 지글지글 끓으면서 아름다운 1,000마일에 달하는 해안선은 수많은 "멋진 탈출구"를 제공합니다. 그 중 일부는 XNUMX월, ...', '캘리포니아. 여름 활동 ; 몬트레이 베이: 고래 관찰 투어. 3 - 4 시간 · (1,382) ; 사우스 레이크 타호: 에메랄드 베이 유람선 관광 크루즈. 2 시간 · (575) ; 몬트레이: ...', '막바지 여름은 쿨하게 해변 ... 막바지 여름을 즐기는 데는 캘리포니아 해변이 최적의 장소다. ... 활동을 즐길 수 있어 여름이면 가족단위의 방문객들이 많이 ...', '로스앤젤레스는 끝없는 여름의 정수를 경험할 수 있는 최고의 도시입니다. 말리부에서 사우스 베이에 이르는 LA 최고의 해변 8곳을 모아봤습니다.', '해변으로 떠나는 완벽한 휴가를 계획 중이라면 캘리포니아주 헌팅턴 비치에 방문해보세요. 자전거를 타고 서핑을 배우고 훌륭한 해산물 요리를 음미하며 멋진 상점에서 ...', '겨울바다의 느낌 속에서도 서퍼들과 자전거와 산책을 즐기는 사람들로 분주한 해변도로는 묘한 대비를 불러 일으킨다. 어느 계절에 와도 좋을 캘리포니아의 전형적인 ...']
Invoking: `write_file` with `{'file_path': 'result_GP

## 3. Tool을 직접 제작해 기능 확장하기
> Tool을 직접 만들어서 Agent가 할 수 있는 일을 늘릴수 있도록 한다. Tool은 다음과 같은 세가지 요소로 셋팅
- 이름 : Tool을 식별하기 위한 이름
- 기능 설명 : Tool이 무엇을 하는지에 대한 간략한 설명, Agent가 Tool을 어떻게 사용할 지 판단할 수 있는 자료
- 실행 함수: Tool이 실제로 작동할 때 처리

In [41]:
'''
Tool을 만들어보자
'''
from langchain.agents import AgentType, Tool, initialize_agent  #←Tool을 가져오기

def min_limit_random_number(min_number): #←원하는 기능을 함수로 정의함
    return random.randint(int(min_number), 100000)

Random = Tool(
                name="Random",  #←도구 이름
                description="특정 최솟값 이상의 임의의 숫자를 생성할 수 있습니다.",  #←도구 설명
                func=min_limit_random_number  #←도구가 실행될 때 호출되는 함수
            )
Random

Tool(name='Random', description='특정 최솟값 이상의 임의의 숫자를 생성할 수 있습니다.', func=<function min_limit_random_number at 0x000002B0FA2C00D0>)

In [42]:
'''
tools 리스트에 필요한 함수를 담고,STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION으로 실행
'''

import random  #←임의의 숫자를 생성하기 위해 필요한 모듈을 가져오기
from langchain.agents import AgentType, Tool, initialize_agent  #←Tool을 가져오기
from langchain.chat_models import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.tools import WriteFileTool

chat = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    anthropic_api_key=os.getenv("Anthropic_API_KEY"),

)

tools = [] #← Tool을 담아줄 list

tools.append(WriteFileTool( 
    root_dir="./"
))

tools.append(Random)


agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True
)

result = agent.run("10 이상의 난수를 생성해 random.txt 파일에 저장하세요.")

print(f"실행 결과: {result}")




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Random",
  "action_input": "10"
}
```


Observation: 36648
Thought:To generate a random number greater than or equal to 10 and save it to a file called random.txt, I will take the following steps:

Action:
```
{
  "action": "write_file",
  "action_input": {
    "file_path": "random.txt",
    "text": "36648",
    "append": false
  }
}
```


Observation: File written successfully to random.txt.
Thought:I have generated a random number greater than or equal to 10 (36648) and saved it to the file 'random.txt' as requested.

Action:
```
{
  "action": "Final Answer",
  "action_input": "A random number greater than or equal to 10 (36648) has been generated and saved to the file 'random.txt'."
}
```

> Finished chain.
실행 결과: A random number greater than or equal to 10 (36648) has been generated and saved to the file 'random.txt'.


## 4. Retrievers를 사용해 문장을 검색하는 Tool 만들기
> Retrievers를 Tool로 변환하여 Agent에서 사용해보자

In [43]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.agents.agent_toolkits import create_retriever_tool  #← create_retriever_tool을 가져오기
from langchain.chat_models import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.retrievers import WikipediaRetriever #←WikipediaRetriever를 가져오기
from langchain.tools import WriteFileTool

chat = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    anthropic_api_key=os.getenv("Anthropic_API_KEY"),

)

# chat = ChatOpenAI(
#     temperature=0,
#     model="gpt-3.5-turbo"
# )

tools = [] 

tools.append(WriteFileTool( 
    root_dir="./result/"
))

retriever = WikipediaRetriever( #←WikipediaRetriever를 초기화
    lang="ko", #←언어를 한국어로 설정
    doc_content_chars_max=500,  #←글의 최대 글자 수를 500자로 설정
    top_k_results=1 #←검색 결과 중 상위 1건을 가져옴
)

tools.append(
    create_retriever_tool(  #←Retrievers를 사용하는 Tool을 생성
        name="WikipediaRetriever",  #←Tool 이름
        description="받은 단어에 대한 Wikipedia 기사를 검색할 수 있다",  #←Tool 설명
        retriever=retriever,  #←Retrievers를 지정
    )
)

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True
)

result = agent.run("스카치 위스키에 대해 Wikipedia에서 찾아보고 그 개요를 한국어로 wiskey.txt 파일에 저장하세요.")

print(f"실행 결과: {result}")




> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "WikipediaRetriever",
  "action_input": {"query": "Scotch whisky"}
}
```


Observation: 위스키(영어: whisky 또는 whiskey, 문화어: 위스끼)는 맥아 및 기타 곡류를 당화 발효시킨 발효주를 증류하여 만든 술이다. 주로 보리, 옥수수, 호밀, 밀 등의 곡물이 원료가 된다. 증류 후에는 나무 통에 넣어 숙성시키는게 보통이다
동방의 증류기술이 중세 십자군 전쟁을 통해 서양에 전래되어 후에 아일랜드를 거쳐 스코틀랜드에 전파되어 위스키가 탄생한 것으로 알려져 있다. 십자군 전쟁에 참여했던 가톨릭 수사들에 의해 아랍의 증류 기술이 유럽에 전해져 증류주를 만들게 된 것이다.
오늘날 위스키의 생산지로 유명한 국가는 스코틀랜드, 아일랜드, 미국, 캐나다, 일본이며 그 외에 대만, 인도, 독일, 호주 등에서도 위스키를 생산한다. 독자적인 위스키 문화를 발달시킨 아일랜드, 미국, 캐나다를 제외한 국가는 관례적으로 스카치 위스키의 제조방식과 분류 규정을 따르는 경향이 있다.
스카치 위스키와 아이리시 위스키는 흔히 원료 곡물에 따라 몰트 위스키, 그레인 위스키, 블랜디드 위스키 등
Thought:Action:
```json
{
  "action": "write_file",
  "action_input": {
    "file_path": "whiskey.txt", 
    "text": "위스키는 맥아 및 기타 곡류를 당화하여 발효시킨 후 증류한 술이다. 주요 원료는 보리, 옥수수, 호밀, 밀 등이다. 증류 후에는 통에 넣어 숙성시킨다. 

스카치 위스키는 스코틀랜드에서 만들어진 위스키로, 독특한 제조법과 분류 기준을 갖고 있다. 스카치 위스키는 원료 곡물에 따라 몰트 위스키, 그레인 위스키, 블렌디드 위스키로 나뉜다. 

- 몰트 위스키: 오직 보리만을 발효하여 만든 위스키
- 

## 5. 문맥에 맞게 답변하는 에이전트 만들기
> Memory모듈을 Agent와 결합하여 사용하면 문맥에 맞는 답변을 만들수있다

In [52]:
from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.chat_models import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferMemory  #←ConversationBufferMemory 가져오기
from langchain.retrievers import WikipediaRetriever
chat = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    anthropic_api_key=os.getenv("Anthropic_API_KEY"),
)

# chat = ChatOpenAI(
#     temperature=0,
#     model="gpt-3.5-turbo"
# )

tools = []

# tools.append(WriteFileTool( 
#     root_dir="./result/"
# ))


retriever = WikipediaRetriever(
    lang="ko",
    doc_content_chars_max=500,
    top_k_results=1
)

tools.append(
    create_retriever_tool(  #←Retrievers를 사용하는 Tool을 생성
        name="WikipediaRetriever",  #←Tool 이름
        description="받은 단어에 대한 Wikipedia 기사를 검색할 수 있다",  #←Tool 설명
        retriever=retriever,  #←Retrievers를 지정
    )
)

memory = ConversationBufferMemory(  #←ConversationBufferMemory를 초기화
    memory_key="chat_history",  #←메모리 키를 설정
    return_messages=True  #←메시지를 반환하도록 설정
)

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  #←Agent의 유형을 대화형으로 변경
    handle_parsing_errors=True,
    memory=memory,  #←Memory를 지정
    verbose=True
)

result = agent.run("스카치 위스키에 대해 Wikipedia에서 찾아보고 그 개요를 한국어로 개요를 정리하세요.") #←Wikipedia에서 찾아보라고 지시
print(f"1차 실행 결과: {result}") #←실행 결과를 표시
result_2 = agent.run("이전 지시를 다시 한번 실행하세요.") #←이전 지시를 다시 실행하도록 지시
print(f"2차 실행 결과: {result_2}") #←실행 결과를 표시



> Entering new AgentExecutor chain...
```json
{
    "action": "WikipediaRetriever",
    "action_input": "스카치 위스키"
}
```
Observation: 스카치 위스키(Scotch whisky)는 스코틀랜드에서 제조되는 위스키이다. 영국에서는 "위스키"(whisky)라고 하면, 특별한 언급이 없는 한 스카치 위스키를 가리킨다. 미국에서는 짧게 줄여 "스카치"라고도 불린다.


== 역사 ==

“To Friar John Cor, by order of the King, to make aqua vitae VIII bolls of malt.” — Exchequer Rolls 1494–95, vol x, p. 487.
위스키는 영국 스코틀랜드 지방에서 4~5세기 무렵에 수도승들이 가지고 들어왔다고 추정되는 증류주 제조술을 바탕으로 제조되어 왔다. 1644년에는 위스키 제조에 세금이 붙기 시작하였는데, 이는 전국적으로 은밀한 위스키 제조를 성행하게 만들었고, 급기야는 1780년 무렵 합법 증류소는 8곳인데 반해 불법 증류소는 400여곳이 될 정도로 불법 위스키 제조가 성행하였다. 이에 1823년, 영국 의회는 "소비세법"을 강화하여 면허를 가
Thought:```json
{
    "action": "Final Answer",
    "action_input": "스카치 위스키는 스코틀랜드에서 제조되는 증류주입니다. 역사적으로는 4~5세기 경 수도승들이 가지고 들어온 증류주 제조술에서 유래되었으며, 1644년부터 위스키 제조에 세금이 부과되기 시작했습니다. 초기에는 불법 제조가 성행했지만, 1823년 '소비세법' 강화로 합법화되었습니다. 스카치 위스키는 대보리, 보리, 또는 몇몇 곡물을 발효시켜 만든 증류주이며, 오크통에서 숙성시킵니다. 주요 산지는 스펜사이드, 아일레이, 하이랜드, 로우랜드, 캄벨타운 등입니다."
}
```

> Finished chain.
1차 실행 결과: 스카치

> Agent 내부에서 언어모델 호출횟수 제한하기
- Agent가 Tool을 사용하여 주어진 Task를 달성하려고 하는데, 어떤 Tool을 사용해도 Task를 완벽하게 처리할 수 없는 경우
    - Tool 실행을 무한 반복하여, 불필요한 과금이 발생할 수 있음
    - max_iterations =3 (3회까지만 실행횟수 제한) 같이 최대 반복횟수를 지정하여 방지

> 특정 사용 사례에 특화한 Tool 정리
- GmailToolkit : 이메일보내기, 검색, 초안작성 등 에이전트 활용하여 지메일을 조작 가능
- PlayWrightBrowserToolkit : 크롬등의 브라우저를 프로그램에서 조작 가능, URL열고, 내용 확인하고, 링크를 클릭
- SQLDatabaseToolkit : SQL사용하여 데이터베이스에서 데이터를 조회, 업데이트,삭제 도구 제공, Agent를 사용하여 SQL문 쉽게 실행
- JiraToolkit : 프로젝트 관리 및 팀 작업 관리를 간소화 하기 위한 소프트웨어인 Jira를 조작하기 위한 Tool 제공